In [1]:
spark.sql(f"DROP DATABASE IF EXISTS audit CASCADE")
spark.sql(f"DROP DATABASE IF EXISTS bronze CASCADE")
spark.sql(f"CREATE DATABASE IF NOT EXISTS audit")
spark.sql(f"CREATE DATABASE IF NOT EXISTS bronze")

NameError: name 'spark' is not defined

In [59]:
if not spark.catalog.tableExists("audit.audit_table"):
        # Create the table if it doesn't exist with timestamps
        schema = StructType([
            StructField("Table_Name", StringType(), True),
            StructField("Progress", StringType(), True),
            StructField("Status", StringType(), True),
            StructField("Start_time", TimestampType(), True),  # Changed to TimestampType
            StructField("End_time", TimestampType(), True)      # Changed to TimestampType
        ])
        empty_df = spark.createDataFrame([], schema)
        empty_df.write.format("delta").mode("overwrite").saveAsTable("audit.audit_table")
        schema = "Table_Name STRING, Progress STRING, Status STRING, Start_time TIMESTAMP, End_time  TIMESTAMP"
        spark.sql(f"CREATE TABLE IF NOT EXISTS audit.audit_table ({schema}) USING DELTA")

if not spark.catalog.tableExists("audit.schema_metadata"):
        # Create the metadata table if it doesn't exist
        schema = "Layer_Name STRING, Table_Name STRING, Schema_Flag STRING, Md5_Hash STRING"
        spark.sql(f"CREATE TABLE IF NOT EXISTS audit.schema_metadata ({schema}) USING DELTA")

In [60]:
%run "/home/jovyan/work/notebooks/json_proc/Audit_Table.ipynb"

In [61]:
def create_tables(filepath,dbname,table_name):
    df = spark.read.format("parquet").load(filepath)
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {dbname}")
    new_column_names = [col.replace(' ', '_').replace('-', '_').replace('(', '').replace(')','').replace('.','').replace('___','_') for col in df.columns]
    df = df.toDF(*new_column_names)
    check_and_store_md5_result = check_and_store_md5(df, "Bronze", "EV_Data", schema_flag ="N")
    if check_and_store_md5_result == True:
        df.write.format("delta").mode("append").saveAsTable(f"{dbname}.{table_name}")
        print("check_and_store_md5_result-True")
        df.limit(10).show()
    if check_and_store_md5_result == False:
        df.write.option("mergeSchema", "true").format("delta").mode("append").saveAsTable(f"{dbname}.{table_name}")
        print("check_and_store_md5_result-False")
        df.limit(10).show()

In [62]:
def create_meta_tables(filepath,dbname,table_name):
    df = spark.read.format("parquet").load(filepath)
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {dbname}")
    new_column_names = [col.replace(' ', '_').replace('-', '_').replace('(', '').replace(')','').replace('.','').replace('___','_') for col in df.columns]
    df = df.toDF(*new_column_names)
    df.write.option("mergeSchema", "true").format("delta").mode("overwrite").saveAsTable(f"{dbname}.{table_name}")
    df.limit(10).toPandas().head()

In [63]:
veh_data_path = "/home/jovyan/work/data/raw/derived/vehicle_data/"
dbname = "bronze"
table_name = "vehicle_data"

# Create tables using function
update_audit_table("EV_Data", progress="Ingest_Bronze", status="In_Progress", start=True)
create_tables(veh_data_path,dbname,table_name)
display(spark.sql(f"SELECT * FROM {dbname}.{table_name} LIMIT 10"))
df = spark.sql(f"SELECT * FROM {dbname}.{table_name} LIMIT 10")
df.limit(10).toPandas().head()

+----------+-------------+-----------+--------------------+--------+
|Table_Name|     Progress|     Status|          Start_time|End_time|
+----------+-------------+-----------+--------------------+--------+
|   EV_Data|Ingest_Bronze|In_Progress|2024-12-06 17:14:...|    null|
+----------+-------------+-----------+--------------------+--------+

+----------+----------+-----------+--------+
|Layer_Name|Table_Name|Schema_Flag|Md5_Hash|
+----------+----------+-----------+--------+
+----------+----------+-----------+--------+

+----------+----------+-----------+--------------------+
|Layer_Name|Table_Name|Schema_Flag|            Md5_Hash|
+----------+----------+-----------+--------------------+
|    Bronze|   EV_Data|          N|99b6b6813c00543c7...|
+----------+----------+-----------+--------------------+

check_and_store_md5_result-True
+------------------+--------------------+--------+----------+------------+----------+------------+----+----------+-----------+-----------+-----+-----------

DataFrame[sid: string, id: string, position: string, created_at: string, created_meta: string, updated_at: string, updated_meta: string, meta: string, VIN_1_10: string, County: string, City: string, State: string, Postal_Code: string, Model_Year: string, Make: string, Model: string, Electric_Vehicle_Type: string, Clean_Alternative_Fuel_Vehicle_CAFV_Eligibility: string, Electric_Range: string, Base_MSRP: string, Legislative_District: string, DOL_Vehicle_ID: string, Vehicle_Location: string, Electric_Utility: string, 2020_Census_Tract: string, Counties: string, Congressional_Districts: string, WAOFM_GIS_Legislative_District_Boundary: string]

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,VIN_1_10,County,...,Electric_Range,Base_MSRP,Legislative_District,DOL_Vehicle_ID,Vehicle_Location,Electric_Utility,2020_Census_Tract,Counties,Congressional_Districts,WAOFM_GIS_Legislative_District_Boundary
0,row-zt4k~iszy.uhv6,00000000-0000-0000-62B4-C1BC527B773A,0,1676414233,None,1676414284,None,{ },5YJ3E1EA8J,San Diego,...,215,0,None,153998050,None,None,06073018509,None,None,None
1,row-5r58~kb8y.789r,00000000-0000-0000-B54E-F27AFFF902F6,0,1676414233,None,1676414284,None,{ },3FA6P0PU7H,Sedgwick,...,21,0,None,138214331,POINT (-97.27013 37.54531),None,20173009801,1291,None,None
2,row-84ix~3wif_u9ju,00000000-0000-0000-F67B-BBFF22B88E48,0,1676414233,None,1676414298,None,{ },1N4AZ0CP8D,Snohomish,...,75,0,38,3129059,POINT (-122.19388 48.15353),PUGET SOUND ENERGY INC,53061052805,3213,2,40
3,row-wiar-siae_sed9,00000000-0000-0000-0360-775CFE2EDAFF,0,1676414233,None,1676414298,None,{ },WBY8P8C58K,Kitsap,...,126,0,26,166525635,POINT (-122.62749 47.565),PUGET SOUND ENERGY INC,53035080500,848,6,33
4,row-abd5~finn.nzkg,00000000-0000-0000-3182-A2040CC92549,0,1676414233,None,1676414298,None,{ },5YJ3E1EA7K,Snohomish,...,220,0,32,475248315,POINT (-122.31768 47.87166),PUGET SOUND ENERGY INC,53061050800,3213,2,7


In [54]:
update_audit_table("EV_Data", progress="Ingest_Bronze", status="Completed", end=True)

+----------+-------------+-----------+--------------------+--------+
|Table_Name|     Progress|     Status|          Start_time|End_time|
+----------+-------------+-----------+--------------------+--------+
|   EV_Data|Ingest_Bronze|In_Progress|2024-12-05 19:11:...|    null|
+----------+-------------+-----------+--------------------+--------+

+----------+-------------+-----------+--------------------+--------+
|Table_Name|     Progress|     Status|          Start_time|End_time|
+----------+-------------+-----------+--------------------+--------+
|   EV_Data|Ingest_Bronze|In_Progress|2024-12-05 19:11:...|    null|
+----------+-------------+-----------+--------------------+--------+

+----------+-------------+---------+--------------------+--------------------+
|Table_Name|     Progress|   Status|          Start_time|            End_time|
+----------+-------------+---------+--------------------+--------------------+
|   EV_Data|Ingest_Bronze|Completed|2024-12-05 19:11:...|2024-12-05 19:

In [55]:
col_meta_path = "/home/jovyan/work/data/raw/derived/columns_metadata/"
table_name = "columns_metadata"
create_meta_tables(col_meta_path,dbname,table_name)
df1 = spark.sql(f"SELECT * FROM {dbname}.{table_name} LIMIT 10")
df1.limit(10).toPandas().head()

,computationStrategy,dataTypeName,description,fieldName,flags,format,id,name,position,renderTypeName,tableColumnId
0,None,meta_data,None,:sid,[hidden],"(None,)",-1,sid,0,meta_data,NaN
1,None,meta_data,None,:id,[hidden],"(None,)",-1,id,0,meta_data,NaN
2,None,meta_data,None,:position,[hidden],"(None,)",-1,position,0,meta_data,NaN
3,None,meta_data,None,:created_at,[hidden],"(None,)",-1,created_at,0,meta_data,NaN
4,None,meta_data,None,:created_meta,[hidden],"(None,)",-1,created_meta,0,meta_data,NaN


In [56]:
tab_meta_path = "/home/jovyan/work/data/raw/derived/table_metadata/"
table_name = "table_metadata"
create_meta_tables(tab_meta_path,dbname,table_name)
df2 = spark.sql(f"SELECT * FROM {dbname}.{table_name} LIMIT 10")
df2.limit(10).toPandas().head()

,assetType,attribution,averageRating,category,createdAt,description,displayType,downloadCount,flags,hideFromCatalog,...,tableAuthor_profileImageUrlLarge,tableAuthor_profileImageUrlMedium,tableAuthor_profileImageUrlSmall,tableAuthor_screenName,tableAuthor_type,approvals_submissionDetails_permissionType,approvals_submissionOutcomeApplication_failureCount,approvals_submissionOutcomeApplication_status,approvals_submitter_displayName,approvals_submitter_id
0,dataset,Washington State Department of Licensing,0,Transportation,1555435581,This dataset shows the Battery Electric Vehicl...,table,52628,default,False,...,/api/users/eagg-6py7/profile_images/LARGE,/api/users/eagg-6py7/profile_images/THUMB,/api/users/eagg-6py7/profile_images/TINY,Department of Licensing,interactive,READ,0,success,Department of Licensing,eagg-6py7
1,dataset,Washington State Department of Licensing,0,Transportation,1555435581,This dataset shows the Battery Electric Vehicl...,table,52628,default,False,...,/api/users/eagg-6py7/profile_images/LARGE,/api/users/eagg-6py7/profile_images/THUMB,/api/users/eagg-6py7/profile_images/TINY,Department of Licensing,interactive,READ,0,success,Department of Licensing,eagg-6py7
2,dataset,Washington State Department of Licensing,0,Transportation,1555435581,This dataset shows the Battery Electric Vehicl...,table,52628,default,False,...,/api/users/eagg-6py7/profile_images/LARGE,/api/users/eagg-6py7/profile_images/THUMB,/api/users/eagg-6py7/profile_images/TINY,Department of Licensing,interactive,READ,0,success,Department of Licensing,eagg-6py7
3,dataset,Washington State Department of Licensing,0,Transportation,1555435581,This dataset shows the Battery Electric Vehicl...,table,52628,default,False,...,/api/users/eagg-6py7/profile_images/LARGE,/api/users/eagg-6py7/profile_images/THUMB,/api/users/eagg-6py7/profile_images/TINY,Department of Licensing,interactive,READ,0,success,Department of Licensing,eagg-6py7
4,dataset,Washington State Department of Licensing,0,Transportation,1555435581,This dataset shows the Battery Electric Vehicl...,table,52628,default,False,...,/api/users/eagg-6py7/profile_images/LARGE,/api/users/eagg-6py7/profile_images/THUMB,/api/users/eagg-6py7/profile_images/TINY,Department of Licensing,interactive,READ,0,success,Department of Licensing,eagg-6py7
